In [1]:
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def markedDepenenceLambda(beta,value):
    p=-(np.log(1-value))/beta
    return p

def markFunction(A,alpha,mark):
    return A*np.exp(alpha*mark)


def function_kernel1(theta,tau):
    kernel=(np.exp(-theta*tau))
    return kernel

def function_kernel2(theta,tau):
    kernel=np.ones(len(tau))*0.01
    return kernel



def simu_multivariateMarked(mus,thetas,betas,T):    
    totalD=len(mus)
    timesteps=[[] for d in range(totalD)]
    l2Marks=[[[]for d in range(totalD)] for d in range(totalD)]
    tmax=0
    while tmax<T:
        lambda1=np.zeros(totalD)
        for d in range(totalD):
            lambda1[d]=mus[d]
            for k in range(totalD):
                if len(timesteps[k])!=0:
                    temp=tmax-np.array(timesteps[k][::-1])
                    temp=temp[temp>=0]
                    l3=np.array(l2Marks[d][k][::-1])
                    l3=l3[:len(temp)]
                    lambda1[d]+=(function_kernel1(thetas[d][k],temp)*l3).sum()
        
        totalLambda=max(lambda1.sum(),0)
        u=np.random.uniform(0,1,1)[0]
        intArr=-np.log(u)/totalLambda
        tmax=tmax+intArr
        u=np.random.uniform(0,1,1)[0]
        lambdaTmax=np.zeros(totalD)
        for d in range(totalD):
            lambdaTmax[d]=mus[d]
            for k in range(totalD):
                if len(timesteps[k])!=0:
                    temp=tmax-np.array(timesteps[k][::-1])
                    temp=temp[temp>0]
                    l3=np.array(l2Marks[d][k][::-1])
                    l3=l3[:len(temp)]
                    lambdaTmax[d]+=(function_kernel1(thetas[d][k],temp)*l3).sum()
                        
            if totalLambda*u<=max(lambdaTmax.sum(),0):
                timesteps[d].append(tmax)
                g_x=np.random.uniform(0.0,1.0,totalD)
                for j in range(totalD):
                    marks=markedDepenenceLambda(betas[j][d],g_x[j])
                    l2Marks[j][d].append(marks)
                break
            
    return [np.array(timesteps[j])for j in range(totalD)],l2Marks

In [3]:
mus=np.array([0.7,0.7])
thetas=[np.array([7.0,8.0]),np.array([9.0,10.0])]
betas=[np.array([7,5]),np.array([7,5])]
t,marks1=simu_multivariateMarked(mus,thetas,betas,3000)
for i in range(len(marks1)):
    for j in range(len(marks1)):
        marks1[i][j]=np.array(marks1[i][j])
len(marks1[0][0]),len(marks1[1][0]),len(marks1[0][1]),len(marks1[1][1]),len(t[0]),len(t[1])

(2181, 2181, 2114, 2114, 2181, 2114)

In [4]:
dictdimP={}
mapping={}
def definitions(t):
    totalD=len(t)
            
    dimensions=np.arange(0,totalD,1)
    for i in range(totalD):
        dictdimP[i]=np.delete(dimensions,i)

    for i in range(totalD):
        for j in (dictdimP[i]):
            mapping[i,j]=createMapAtoBIndex(t[i],t[j])
    tmax=0
    for p in range(totalD):
        tmax=max(tmax,t[p][-1])
    return tmax
    
def loglikelihoodMulti(t,marks,mus,thetas,betas):
    tmax = definitions(t)
    totalD=len(t)

  
    ll=0
    integrated=0
    for p in range(0,totalD,1):
        a=0
        tend = (t[p][-1]-t[p][:])
        a+=mus[p]*(t[p][-1]-t[p][0])
        a+=(anyRandomIntegratedKernel(tend,marks[p][p],thetas[p][p])).sum()
    
        for k in dictdimP[p]:
            tend=(t[p][-1]-t[k][:])
            tend=tend[tend>=0]
            marks12=marks[p][k][0:len(tend)]
            a+=(anyRandomIntegratedKernel(tend,marks12,thetas[p][k])).sum()
        integrated+=a
        ll = ll+a
        ll = ll-np.log(mus[p])
        tp = t[p][:]  
        for i in range(1,len(tp),1):
            temp1 = tp[i]-tp[0:i]
            mark=marks1[p][p][0:i]
            decayFactor = (anyKernel(temp1,mark,thetas[p][p])).sum()
            for k in dictdimP[p]:
                jT = mapping[p,k].get(tp[i])
                if(jT != None):
                    j = jT
                    temp2 = tp[i]-t[k][0:j+1]
                    mark2=marks1[p][k][0:j+1]
                    decayFactor = decayFactor + (anyKernel(temp2,mark2,thetas[p][k])).sum()
            logLam = -np.log(mus[p]+decayFactor)
            ll = ll+logLam
    
        for k in range(totalD):
            markLL=(np.log(betas[p][k]*np.exp(-betas[p][k]*marks[p][k]))).sum()
            ll-=markLL
        
    print("Integrated Kernel",integrated)
        
    return ll

def anyRandomIntegratedKernel(tend,mark,theta):

    res = (1-np.exp(-theta*tend))/theta*mark
    
    return res

def anyKernel(tp,mark,theta):
    res = mark*np.exp(-theta*tp)
  
    return res

def createMapAtoBIndex(a,b):
    mapAtoBIndex={}
    p1=0
    for x in range(len(a)):
        y=a[x]
        p=max(0,x)
        b1=b[p1:]
        if(max(b1[b1<y],default=-1)==-1):
            mapAtoBIndex[y] = None
        else:
            p1= p1+(np.where(b1==max(b1[b1<y])))[0][0]
            mapAtoBIndex[y] = p1    

    return mapAtoBIndex

loglikelihoodMulti(t,marks1,mus,thetas,betas)

Integrated Kernel 4372.824222824466


-951.9607343202129

In [ ]:
def gradientIntegratedKernel(tend,mark,p,k):
    theta=thetas[p,k]
    fac1=np.exp(-theta*tend)
    fac2= (mark/theta**2)*(fac1*tend*theta-(1-fac1))
    return fac2.sum()
def gradientKernel(temp,mark,p,k):
    theta=thetas[p,k]
    fac1=-np.exp(-theta*temp)*mark*temp
    return (fac1).sum()
    
def gradientMarks(mark,p,k):
    beta=betas[p,k]
    fac=1/(beta*np.exp(-beta*mark))
    grad= fac*np.exp(-beta*mark)*(-beta*mark+1)

    return grad
    



def gradientll(iArray,marks):

    gradthetas = np.zeros([totalD,totalD])
    gradbetas= np.zeros([totalD,totalD])
    grad_mu=np.zeros(totalD)
    for i in range(0,max(iArray.shape),1):
        factor1=0
        p = int(iArray[0,i])

        index = int(iArray[1,i])
        if index>0:
            tend = (tmax-t[p][index])
            mark=marks[p][p][index]
            grad1=gradientIntegratedKernel(tend,mark,p,p)
            gradthetas[p,p]=gradthetas[p,p]+grad1
            temp1 = t[p][index]-t[p][0:index]
            mark1=marks[p][p][0:index]
            decayFactor = (function_kernel1(thetas[p,p],temp1)*mark1).sum()
            factor1=  gradientKernel(temp1,mark1,p,p)
            gradMarks1=gradientMarks(mark,p,p)
            factor2=np.zeros(totalD)
            for k in (dictdimP[p]):
                jT=mapping[p,k].get(t[p][index])
                if (jT!=None):
                    if jT>len(t[k])-1:
                        jT=t[k][-1]
                    tend=tmax-t[k][jT]
                    mark=marks[p][k][jT]
                    grad2=gradientIntegratedKernel(tend,mark,p,k)
                    gradthetas[p,k]=gradthetas[p,k]+grad2
                    gradMarks2=gradientMarks(mark,p,k)
                    gradbetas[p,k] = gradbetas[p,k] - gradMarks2
                    temp2 = t[p][index]-t[k][0:jT+1]
                    mark2=marks[p][k][0:jT+1]
                    decayFactor = decayFactor +(function_kernel1(thetas[p,k],temp2)*mark2).sum()
                    factor2[k] = gradientKernel(temp2,mark2,p,k)
            lam = mus[p]+decayFactor
            gradthetas[p,p] = gradthetas[p,p] - (1/lam)*factor1
            gradbetas[p,p] = gradbetas[p,p] - gradMarks1
            for k in (dictdimP[p]):
                if factor2[k]!=0 :
                    gradthetas[p,k] = gradthetas[p,k] -(1/lam)*factor2[k]
            grad_mu[p]=grad_mu[p]+((t[p][index]-t[p][index-1])-(1/lam))*(index >1)
                             
    return gradthetas/max(iArray.shape),gradbetas/max(iArray.shape),grad_mu/max(iArray.shape)

def sgdHawkes(nEpochs,lr,t,marks):
    global bestll,thetas,betas,mus
    lr = lr
    beta_1 = 0.9
    beta_2 =0.999
    epsilon = 1e-8
    count = 0
    bestll = 1e8
    best_thetas=thetas
    best_betas=betas
    best_mus=mus
    lr_mu=lr

    m_tthetas = np.zeros([totalD,totalD])
    v_tthetas = np.zeros([totalD,totalD])
    m_tbetas = np.zeros([totalD,totalD])
    v_tbetas = np.zeros([totalD,totalD])
    m_t_mu= np.zeros(totalD)
    v_t_mu = np.zeros(totalD)
    totalLength=0
    for j in range(len(t)):
        totalLength+=len(t[j])
    tCompressed=np.zeros((2,totalLength))
    length=0
    for j in range(len(t)):
        length1=length
        length+=len(t[j])
        tCompressed[0,length1:length]=j
        tCompressed[1,length1:length]=np.arange(0,length-length1,1)
    print(totalLength,"number of timepoints")
    
    for epochs in range(1,nEpochs,1):
        
        rsample = np.random.choice(totalLength,totalLength,replace = False)
        for i in range(0,len(rsample),32):
            count=count+1 
            gradthetas,gradbetas,grad_mu = gradientll(tCompressed[:,rsample[i:i+32]],marks)
            
            m_tthetas = beta_1*m_tthetas + (1-beta_1)*gradthetas	#updates the moving averages of the gradient
            v_tthetas = beta_2*v_tthetas + (1-beta_2)*(gradthetas*gradthetas)	#updates the moving averages of the squared gradient
            m_capthetas = m_tthetas/(1-(beta_1**count))		#calculates the bias-corrected estimates
            v_capthetas = v_tthetas/(1-(beta_2**count))		#calculates the bias-corrected estimates
            thetas = thetas-(lr*m_capthetas)/(np.sqrt(v_capthetas)+epsilon)
            thetas = np.maximum(thetas,0)
            
            
            m_tbetas = beta_1*m_tbetas + (1-beta_1)*gradbetas	#updates the moving averages of the gradient
            v_tbetas = beta_2*v_tbetas + (1-beta_2)*(gradbetas*gradbetas)	#updates the moving averages of the squared gradient
            m_capbetas = m_tbetas/(1-(beta_1**count))		#calculates the bias-corrected estimates
            v_capbetas = v_tbetas/(1-(beta_2**count))		#calculates the bias-corrected estimates
            betas = betas-(lr*m_capbetas)/(np.sqrt(v_capbetas)+epsilon)
            betas = np.maximum(betas,0)
           
            
            m_t_mu = beta_1*m_t_mu + (1-beta_1)*grad_mu	#updates the moving averages of the gradient
            v_t_mu = beta_2*v_t_mu + (1-beta_2)*(grad_mu*grad_mu)	#updates the moving averages of the squared gradient
            m_cap_mu = m_t_mu/(1-(beta_1**count))		#calculates the bias-corrected estimates
            v_cap_mu = v_t_mu/(1-(beta_2**count))		#calculates the bias-corrected estimates
            mus = mus-(lr_mu*m_cap_mu)/(np.sqrt(v_cap_mu)+epsilon)
            mus = np.maximum(mus,0)
        
        error=loglikelihoodMulti(t,marks,mus,thetas,betas)

        best_betas=best_betas*(bestll<error)+betas*(bestll>error)
        best_thetas=best_thetas*(bestll<error)+thetas*(bestll>error)
        best_mus=best_mus*(bestll<error)+mus*(bestll>error)
        
        bestll = min(bestll,error)
        
           
        print(epochs,i,error, bestll,'\n',thetas,betas,"\n",mus)   #iteration, -loglikelihood, bestloglik, currentpara, bestpara  
    
    return best_thetas,best_betas,best_mus,bestll
    
lr=0.005

totalD=len(t)
tmax=0
for p in range(totalD):
    tmax=max(t[p][-1],tmax)
thetas=np.random.uniform(0,1,totalD**2).reshape(totalD,totalD)
#thetas=np.array([np.array([0.8,0.8]),np.array([1.0,1.0])]).reshape(2,2)
betas=np.random.uniform(0,1,totalD**2).reshape(totalD,totalD)
#betas=np.array([np.array([1.5,4.0]),np.array([8.0,2.0])]).reshape(2,2)

baseline = np.random.uniform(0,1,totalD)
mus = baseline
#mus=np.array([0.7,0.7])
sgdHawkes(300,lr,t,marks1)

4295 number of timepoints
Integrated Kernel 4706.723417833915
1 4288 5526.974743679754 5526.974743679754 
 [[0.8667611  0.47162614]
 [0.33590979 0.97216016]] [[1.48926507 1.44166251]
 [1.3435408  1.04133391]] 
 [0.37229649 0.32158062]
Integrated Kernel 4551.190592714911
2 4288 3648.8764814646593 3648.8764814646593 
 [[0.97425857 0.56491588]
 [0.37440132 1.15193692]] [[1.95078479 1.88392341]
 [1.80076283 1.44735186]] 
 [0.40701887 0.35242686]
Integrated Kernel 4577.253429166078
3 4288 2505.5714033289273 2505.5714033289273 
 [[1.06733895 0.65252181]
 [0.41727094 1.29446689]] [[2.35634748 2.25823885]
 [2.18951193 1.78644318]] 
 [0.45214332 0.40102739]
Integrated Kernel 4598.820813516239
4 4288 1725.9755814162977 1725.9755814162977 
 [[1.15208303 0.73656928]
 [0.47657452 1.43228781]] [[2.71698534 2.5807779 ]
 [2.53949296 2.0872675 ]] 
 [0.50645747 0.42733626]
Integrated Kernel 4578.470934997918
5 4288 1161.062753096135 1161.062753096135 
 [[1.25963449 0.81654872]
 [0.52211092 1.54994274]] 

Integrated Kernel 4281.652304260813
40 4288 -933.477784971205 -933.477784971205 
 [[3.6730689  2.87830017]
 [2.27599305 4.83541098]] [[7.00481055 4.97218232]
 [6.85069903 4.89731434]] 
 [0.63925014 0.63605343]
Integrated Kernel 4337.352695104905
41 4288 -934.8200184505927 -934.8200184505927 
 [[3.72952114 2.93032058]
 [2.33245847 4.91770082]] [[6.99596599 4.98066993]
 [6.85185782 4.90508997]] 
 [0.65531236 0.64146466]
Integrated Kernel 4284.967588351436
42 4288 -935.4014611720006 -935.4014611720006 
 [[3.78658669 2.99276095]
 [2.38576452 5.00245525]] [[7.01055912 4.96641234]
 [6.87309487 4.90730256]] 
 [0.65134659 0.63087184]
Integrated Kernel 4281.949190040152
43 4288 -936.0391186980737 -936.0391186980737 
 [[3.83439223 3.05423678]
 [2.4414413  5.09376606]] [[7.00230674 4.96492414]
 [6.86817442 4.8976693 ]] 
 [0.6537121 0.630298 ]
Integrated Kernel 4218.78080208997
44 4288 -935.6453623658347 -936.0391186980737 
 [[3.9074036  3.1205077 ]
 [2.50427782 5.17553091]] [[7.00504646 4.9596364

Integrated Kernel 4357.720075183681
79 4288 -949.4572383573786 -949.4572383573786 
 [[5.41843593 5.661401  ]
 [4.9215583  7.77363519]] [[6.99763538 4.98115659]
 [6.88567568 4.92398093]] 
 [0.69487004 0.67478824]
Integrated Kernel 4377.164856115095
80 4288 -949.4131241105979 -949.4572383573786 
 [[5.42274598 5.76053501]
 [5.02052913 7.83547773]] [[7.0097053  4.95780811]
 [6.90129505 4.91736385]] 
 [0.69373782 0.68341593]
Integrated Kernel 4379.8682171938435
81 4288 -949.400016094698 -949.4572383573786 
 [[5.46552642 5.8307257 ]
 [5.06817293 7.92075455]] [[7.02613797 4.95862675]
 [6.88657516 4.90089709]] 
 [0.69463033 0.68426215]
Integrated Kernel 4234.0127824184165
82 4288 -949.5479654064463 -949.5479654064463 
 [[5.50804579 5.91110465]
 [5.17511431 7.96123215]] [[7.01405005 4.95664441]
 [6.87583239 4.89525546]] 
 [0.67672926 0.65450521]
Integrated Kernel 4220.721605324881
83 4288 -949.2918240743636 -949.5479654064463 
 [[5.53605567 5.98462685]
 [5.26447043 8.01457465]] [[7.02618195 4.9

Integrated Kernel 4245.749976055378
118 4288 -952.159214243489 -952.2249183345907 
 [[6.12137472 8.01024819]
 [7.7011579  9.67605037]] [[7.01170929 4.9498625 ]
 [6.88257487 4.904784  ]] 
 [0.68589744 0.66718522]
Integrated Kernel 4321.281485950858
119 4288 -952.175795842053 -952.2249183345907 
 [[6.13957077 8.0315751 ]
 [7.73196315 9.70516713]] [[7.00078864 4.96364522]
 [6.88786665 4.90540062]] 
 [0.68761393 0.69086336]
Integrated Kernel 4327.2397253492745
120 4288 -952.2974791918625 -952.2974791918625 
 [[6.1467884  8.05961008]
 [7.79920768 9.73439225]] [[7.01620953 4.93912987]
 [6.87418443 4.90654609]] 
 [0.70014946 0.68054754]
Integrated Kernel 4230.335842446739
121 4288 -948.2712961821778 -952.2974791918625 
 [[6.16064538 8.12232446]
 [7.85491065 9.77920937]] [[7.00848811 4.9609076 ]
 [6.87592749 4.90193025]] 
 [0.65182581 0.69691221]
Integrated Kernel 4303.255210777633
122 4288 -952.2646371853234 -952.2974791918625 
 [[6.15469477 8.12368787]
 [7.89559636 9.82750626]] [[7.01115197 

Integrated Kernel 4481.165601155835
157 4288 -948.2173142950237 -953.0033531129181 
 [[ 6.35454689  9.17755577]
 [ 9.39805133 10.79570696]] [[6.99753108 4.9577409 ]
 [6.89052896 4.90416549]] 
 [0.7119484  0.72650216]
Integrated Kernel 4297.40775651432
158 4288 -952.812703877383 -953.0033531129181 
 [[ 6.36043692  9.21596917]
 [ 9.44424931 10.82664452]] [[7.01273407 4.97988521]
 [6.87557929 4.91259857]] 
 [0.68968552 0.687644  ]
Integrated Kernel 4282.963949763817
159 4288 -952.5731778542031 -953.0033531129181 
 [[ 6.35352789  9.24986888]
 [ 9.45766419 10.86483233]] [[7.01299504 4.94347903]
 [6.88826211 4.92522972]] 
 [0.70208342 0.67051844]
Integrated Kernel 4223.9610338227485
160 4288 -952.516994019513 -953.0033531129181 
 [[ 6.38358656  9.27684408]
 [ 9.51528685 10.88398018]] [[7.01822825 4.96092146]
 [6.90772778 4.91786234]] 
 [0.6850096  0.66813047]
Integrated Kernel 4327.316034463629
161 4288 -952.1526444841988 -953.0033531129181 
 [[ 6.37543312  9.31058724]
 [ 9.50278512 10.90644

Integrated Kernel 4329.924348923341
195 4288 -952.6316831336076 -953.0099097120872 
 [[ 6.47142869  9.83165041]
 [10.28459807 11.50785518]] [[7.01566838 4.97500591]
 [6.88149512 4.90910643]] 
 [0.69405565 0.69704482]
Integrated Kernel 4388.885410865696
196 4288 -952.0107558324189 -953.0099097120872 
 [[ 6.45329083  9.82289095]
 [10.30743252 11.50926979]] [[7.02349616 4.95414833]
 [6.89019839 4.90402038]] 
 [0.71275171 0.69797244]
Integrated Kernel 4312.661944876752
197 4288 -952.9236705218041 -953.0099097120872 
 [[ 6.47802757  9.83732339]
 [10.31702211 11.54387096]] [[6.99418892 4.96295695]
 [6.87214941 4.90359267]] 
 [0.70124368 0.68418719]
Integrated Kernel 4231.02132825879
198 4288 -952.4472433814985 -953.0099097120872 
 [[ 6.49563108  9.88459031]
 [10.34204543 11.54950912]] [[6.99091137 4.94853703]
 [6.88077266 4.90209308]] 
 [0.68049249 0.677858  ]
Integrated Kernel 4362.24686648002
199 4288 -952.4378497319003 -953.0099097120872 
 [[ 6.48384733  9.86136133]
 [10.36022704 11.57985

Integrated Kernel 4321.003479282758
233 4288 -952.8969736694582 -953.0099097120872 
 [[ 6.41284929 10.11200958]
 [10.85214718 11.99930169]] [[7.00875341 4.9530729 ]
 [6.87896925 4.90609928]] 
 [0.70420391 0.68520431]
Integrated Kernel 4436.016937131687
234 4288 -950.4769325355555 -953.0099097120872 
 [[ 6.43258398 10.12381437]
 [10.84632391 11.99516015]] [[6.99218179 4.95343101]
 [6.88365782 4.87568344]] 
 [0.71360512 0.71422604]
Integrated Kernel 4343.362642275428
235 4288 -952.8047524827898 -953.0099097120872 
 [[ 6.4335553  10.12881751]
 [10.87052487 12.01544611]] [[7.00328712 4.96601549]
 [6.8895899  4.88894211]] 
 [0.70594999 0.69102446]
Integrated Kernel 4278.087283430087
236 4288 -952.3636774406673 -953.0099097120872 
 [[ 6.43031883 10.15859258]
 [10.8960939  12.02824525]] [[7.02183949 4.94890912]
 [6.8681826  4.90571678]] 
 [0.70522697 0.67003738]
Integrated Kernel 4222.774740570881
237 4288 -951.866124553929 -953.0099097120872 
 [[ 6.43225028 10.1602517 ]
 [10.92760987 12.0250

Integrated Kernel 4287.5897727494175
271 4288 -953.0793471121544 -953.0793471121544 
 [[ 6.45431911 10.3266249 ]
 [11.19007378 12.33083555]] [[7.00445824 4.97058373]
 [6.89018648 4.91754619]] 
 [0.69232049 0.68695426]
Integrated Kernel 4237.981254013688
272 4288 -952.7713494307009 -953.0793471121544 
 [[ 6.45639265 10.34815022]
 [11.2143373  12.35358332]] [[7.00851384 4.97681087]
 [6.89923622 4.91035441]] 
 [0.68327979 0.67952552]
Integrated Kernel 4334.751081925134
273 4288 -951.3597222971403 -953.0793471121544 
 [[ 6.43978036 10.31264259]
 [11.2173058  12.35746158]] [[7.01024692 4.9627349 ]
 [6.90313634 4.91728467]] 
 [0.72333199 0.67164698]
Integrated Kernel 4215.24655468096
274 4288 -952.3250818301613 -953.0793471121544 
 [[ 6.44939476 10.35626016]
 [11.22231971 12.38343966]] [[7.01550575 4.95788426]
 [6.87986638 4.9053402 ]] 
 [0.6859     0.66934598]
Integrated Kernel 4316.0824718747135
275 4288 -952.8709144710319 -953.0793471121544 
 [[ 6.46061352 10.35385241]
 [11.23200278 12.37

In [ ]:
mapping[0,1]

In [ ]:
s=np.array([0,2,4,6,7,8])
s[-2:]